In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
from keras.applications.densenet import preprocess_input
from keras.preprocessing import image
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow import keras
from keras.layers import Input, Flatten, Dense, Dropout, Conv2D, AveragePooling2D, Concatenate, GlobalAveragePooling2D
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, AveragePooling2D, GlobalMaxPooling2D, BatchNormalization, MaxPooling2D
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import plot_roc_curve, classification_report, plot_confusion_matrix, accuracy_score
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
from keras.layers import BatchNormalization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def save_dump(file_path, data, labels):
    file = open(file_path, 'wb')
    # dump information to that file
    pickle.dump((data, labels), file)
    # close the file
    file.close()
    pass


def load_data(path_file):
    file = open(path_file, 'rb')
    # dump information to that file
    (pixels, labels) = pickle.load(file)
    # close the file
    file.close()
    print(pixels.shape)
    print(labels.shape)
    return pixels, labels


def view_chart(performance, people, chart):
    fig, ax = plt.subplots()
    y_pos = np.arange(len(people))
    ax.barh(y_pos, performance, align='center', color=['green', 'yellowgreen', 'dodgerblue','orange'])
    for index, value in enumerate(performance):
        plt.text(value, index, str(value))
    ax.set_yticks(y_pos)
    ax.set_yticklabels(people)
    ax.invert_yaxis()
    ax.set_xlabel('Number')
    ax.set_title(chart)
    plt.xlim(0, max(performance) + 400)
    plt.show()
    pass


def view_model_history(model_history):
    # view
    accuracy = model_history.history['accuracy']
    val_accuracy = model_history.history['val_accuracy']
    loss = model_history.history['loss']
    val_loss = model_history.history['val_loss']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Training accuracy')
    plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()
    pass


def get_labels_smids(labels_name):
    labels_number = []
    for lb in labels_name:
        if lb == '01-Normal':
            labels_number.append(0)
        elif lb == '02-Tapered':
            labels_number.append(1)
        elif lb == "03-Pyriform":
            labels_number.append(2)
        elif lb == "04-Small":
            labels_number.append(3)
        else:
            labels_number.append(4)
    return np.array(labels_number)

In [ ]:
# Dataset SCIAN
CATEGORIES = ["01-Normal", "02-Tapered", "03-Pyriform", "04-Small", "05-Amorphous"]
scian_train = '/content/drive/MyDrive/human_sperm_morphology_dataset/SCIAN-Morpho/dataset/scian_train.data'
scian_valid = '/content/drive/MyDrive/human_sperm_morphology_dataset/SCIAN-Morpho/dataset/scian_valid.data'
scian_test = '/content/drive/MyDrive/human_sperm_morphology_dataset/SCIAN-Morpho/dataset/scian_datatest.data'

data_scian_train, labels_scian_train = load_data(scian_train)
data_scian_valid, labels_scian_valid = load_data(scian_valid)
data_scian_test, labels_scian_test = load_data(scian_test)

lb = LabelBinarizer()
labels_scian_train_one_hot = lb.fit_transform(labels_scian_train)
labels_scian_valid_one_hot = lb.fit_transform(labels_scian_valid)
labels_scian_test_one_hot = lb.fit_transform(labels_scian_test)

labels_scian_train_one_hot.shape, labels_scian_valid_one_hot.shape, labels_scian_test_one_hot.shape

(9125, 33, 33, 3)
(9125,)
(2282, 33, 33, 3)
(2282,)
(2852, 33, 33, 3)
(2852,)


((9125, 5), (2282, 5), (2852, 5))

In [ ]:
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(33, 33, 3))
# for layer in base_model.layers:
#     layer.trainable = False

# # Create new model.
# headModel = base_model.output
# headModel = BatchNormalization()(headModel)
# headModel = GlobalAveragePooling2D()(headModel)
# headModel = Dropout(0.4)(headModel)
# headModel = Dense(1024, activation='relu')(headModel)
# headModel = Dropout(0.5)(headModel)
# headModel = Dense(5, activation='softmax')(headModel)

# model = Model(inputs=base_model.input, outputs=headModel)
# model.summary()

In [ ]:
input_shape=(33, 33, 3)

model = tf.keras.Sequential()
model.add(Conv2D(filters=64, kernel_size=(2, 2), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()

In [ ]:
# model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     metrics=[tf.keras.metrics.CategoricalCrossentropy(name="accuracy")],
# )

model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalCrossentropy(name="accuracy")])

In [ ]:
size = 64
ep = 50
ver = 1

model_history = model.fit(data_scian_train, labels_scian_train_one_hot, batch_size=size, epochs=ep, verbose=ver, 
                          validation_data=(data_scian_valid, labels_scian_valid_one_hot), shuffle=True)

view_model_history(model_history)

In [ ]:
test_eval = model.evaluate(data_scian_test, labels_scian_test_one_hot, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

90/90 [==============================] - 2s 19ms/step - loss: 1.9205 - accuracy: 0.4309
Test loss: 1.9205259084701538
Test accuracy: 0.4309256672859192


In [ ]:
y_predict = model.predict(data_scian_test)
y_target = np.argmax(y_predict, axis=1)
labels_scian_test_number = get_labels_smids(labels_scian_test)

print("Accuracy: %.2f %%" % (100 * accuracy_score(labels_scian_test_number, y_target)))
print(confusion_matrix(labels_scian_test_number, y_target))

target_names = ["Class {}".format(i) for i in range(len(np.unique(labels_scian_test_number)))]
print(classification_report(labels_scian_test_number, y_target, target_names=target_names))

Accuracy: 43.09 %
[[308 137  84  42  37]
 [121 441  70  36  43]
 [142 147 150  20  35]
 [144  85  30 159  41]
 [120 134  72  83 171]]
              precision    recall  f1-score   support

     Class 0       0.37      0.51      0.43       608
     Class 1       0.47      0.62      0.53       711
     Class 2       0.37      0.30      0.33       494
     Class 3       0.47      0.35      0.40       459
     Class 4       0.52      0.29      0.38       580

    accuracy                           0.43      2852
   macro avg       0.44      0.41      0.41      2852
weighted avg       0.44      0.43      0.42      2852



In [ ]:
model.save("/content/drive/MyDrive/human_sperm_morphology_dataset/models/smids_resnet50.h5")